In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import Merge, TimeDistributed, concatenate, Bidirectional, Masking, RepeatVector
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.preprocessing.sequence import pad_sequences
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.callbacks import Callback
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU, Conv2D
from keras.callbacks import Callback, History
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D, Masking
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

/usr/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
# import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler,MinMaxScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

In [3]:
train= pd.read_csv('train.csv')
test= pd.read_csv('test_BDIfz5B.csv')
campaign_df= pd.read_csv('campaign_data.csv')

In [4]:
train_df= train.merge(campaign_df,how='left')
test_df= test.merge(campaign_df,how='left')

In [5]:
train_df.send_date=pd.to_datetime(train_df.send_date,format="%d-%m-%Y %H:%M")

In [6]:
# Form sentences for clicks per user, open per user, etc
sentences=train_df.groupby('user_id').is_click.apply(lambda x: list(x))

In [7]:
sentences_open = train_df.groupby('user_id').is_open.apply(lambda x: list(x))

In [8]:
train_df['communication_id'],invercom= pd.factorize(train_df.communication_type)

In [9]:
sentences_days = train_df.groupby('user_id').no_of_sections.apply(lambda x: list(x))

In [10]:
sentences_com = train_df.groupby('user_id').no_of_images.apply(lambda x: list(x))

In [11]:
sentences_links = train_df.groupby('user_id').total_links.apply(lambda x: list(x))

In [12]:
sennew=zip(sentences, sentences_open,sentences_days,sentences_com,sentences_links)

In [13]:
# make n_rows*n_timesteps* n_features vector for all sequences
sequences_ori=pad_sequences(pd.Series(sennew).apply(lambda x: zip(x[0],x[1])).tolist(),value=-1,padding='pre')

In [14]:
# filter those which have length of 1

sentences2=sentences[sentences.apply(lambda x: len(x))>1]

In [15]:
sentences_open2 = sentences_open[sentences_open.apply(lambda x: len(x))>1]

In [16]:
sentences_days2= sentences_days[sentences_days.apply(lambda x: len(x)>1)]

In [17]:
sentences_com2= sentences_com[sentences_com.apply(lambda x: len(x)>1)]

In [18]:
sentences_links2= sentences_links[sentences_links.apply(lambda x: len(x)>1)]

In [19]:
sennew=zip(sentences2, sentences_open2,sentences_days2,sentences_com2,sentences_links2)
# seqser=pd.concat((pd.Series(sennew).apply(lambda x: zip(x[0],x[1])).apply(lambda x: random.sample(x,len(x))),
# pd.Series(sennew).apply(lambda x: zip(x[0],x[1]))))
seqser=pd.Series(sennew).apply(lambda x: zip(x[0],x[1]))
sequences=pad_sequences(seqser.tolist(),value=-1,padding='pre')

In [149]:
sequences.shape

(151470, 20, 2)

In [150]:
sequences_ori.shape

(168236, 20, 2)

In [20]:
train_X= sequences[:,:-1]
train_Y = sequences[:,-1]

In [21]:
train_Y= train_Y[:,0]

In [178]:
train_X.shape

(151470, 19, 4)

In [22]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

In [30]:

# define nn model
# emdedding_size=500
# vocab_size=5502
keras_model2 = Sequential()
keras_model2.add(Masking(mask_value=-1. ,input_shape=(None,2)))
# keras_model2.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, 
#                     weights=[pretrained_weights]))
# keras_model2/.add(LSTM(units=100,return_sequences=True))
# keras_model2.add(Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform",input_shape=(None,4)))
# keras_model2.add(MaxPooling1D())
# print keras_model2.output_shape
# keras_model2.add(Conv1D(300, 3, padding='valid',activation='relu',strides=1 ,input_shape=(None,4)))
# keras_model2.add(Conv1D(150, 3, padding='valid',activation='relu',strides=2))
# keras_model2.add(Conv1D(75, 3, padding='valid',activation='relu',strides=2))
# keras_model2.add(Flatten())
# keras_model2.add(Dropout(0.2))
# model.add(Dense(150,activation='sigmoid'))
# model.add(Dropout(0.2))
# keras_model2.add(Conv1D(100,
#                  3,
#                  padding='valid',
#                  activation='relu',
#                  strides=1,input_shape=(None,4)))
# we use max pooling:
# keras_model2.add(GlobalMaxPooling1D())
keras_model2.add((LSTM(units=100,return_sequences=False,recurrent_dropout=0.2)))
# keras_model2.add((GRU(units=100,return_sequences=False,recurrent_dropout=0.2)))
# keras_model2.add(Conv1D(300, 3, padding='valid',activation='relu',strides=2 ,input_shape=(None,4)))
# keras_model2.add(Conv1D(150, 3, padding='valid',activation='relu',strides=2))
# keras_model2.add(Conv1D(75, 3, padding='valid',activation='relu',strides=2))
# keras_model2.add(Flatten())
print keras_model2.output_shape
# keras_model2.add(Conv1D(32, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform"))
# print keras_model2.output_shape
# keras_model2.add(GlobalMaxPooling1D())
print keras_model2.output_shape

# keras_model2.add(Dropout(0.2))
# keras_model2.add(LSTM(units=500,return_sequences=False))
keras_model2.add(Dense(units=50))
keras_model2.add(Dropout(0.2))
keras_model2.add(Dense(units=10))
keras_model2.add(Dropout(0.2))
# keras_model2.add(Dense(units=5))
# keras_model2.add(Dropout(0.2))

keras_model2.add(Dense(units=1))
print keras_model2.output_shape
keras_model2.add(Activation('sigmoid'))
keras_model2.compile(optimizer='Adamax', loss='binary_crossentropy',metrics=['accuracy','mse'])

(None, 100)
(None, 100)
(None, 1)


In [31]:
# train_X=np.concatenate((np.ones_like(train_X[:,0]).reshape(train_X.shape[0],-1,4)*-1,train_X),axis=1)

In [32]:
xtrain, xval, ytrain, yval = train_test_split(train_X, train_Y, test_size=0.1, random_state=7)

In [33]:
train_X.shape, xtrain.shape, xval.shape

((151470, 19, 2), (136323, 19, 2), (15147, 19, 2))

In [34]:
# filepath="../input/best-model/best.hdf5"
import pickle
filename_m= 'lstm_'
pickle.dump(keras_model2.to_json(),open(filename_m+'.pkl','w'))
filepath=filename_m+"weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='max')
ra_val = RocAucEvaluation(validation_data=(xval, yval), interval = 1)
callbacks_list = [checkpoint,ra_val]

In [36]:
xtrain.shape

(136323, 19, 2)

In [37]:
keras_model2.fit(xtrain,ytrain, epochs=29, validation_data=(xval,yval),
          batch_size=1024, callbacks = callbacks_list,verbose=1)

Train on 136323 samples, validate on 15147 samples
Epoch 1/29
136192/136323 [============================>.] - ETA: 0s - loss: 0.1349 - acc: 0.9877 - mean_squared_error: 0.0282Epoch 00000: saving model to lstm_weights-improvement-00-0.99.hdf5

 ROC-AUC - epoch: 1 - score: 0.659234
136323/136323 [==============================] - 56s - loss: 0.1348 - acc: 0.9877 - mean_squared_error: 0.0282 - val_loss: 0.0660 - val_acc: 0.9877 - val_mean_squared_error: 0.0121
Epoch 2/29
136192/136323 [============================>.] - ETA: 0s - loss: 0.0688 - acc: 0.9879 - mean_squared_error: 0.0123Epoch 00001: saving model to lstm_weights-improvement-01-0.99.hdf5

 ROC-AUC - epoch: 2 - score: 0.682699
136323/136323 [==============================] - 56s - loss: 0.0688 - acc: 0.9879 - mean_squared_error: 0.0123 - val_loss: 0.0628 - val_acc: 0.9878 - val_mean_squared_error: 0.0118
Epoch 3/29
136192/136323 [============================>.] - ETA: 0s - loss: 0.0665 - acc: 0.9879 - mean_squared_error: 0.0120

KeyboardInterrupt: 

In [29]:
  from keras.models import model_from_json
# keras_model2= model_from_json(pickle.load(open('lstm2fea100.pkl')))
# keras_model2.load_weights('weigths/LSTM2feaadamax5010batch1024_recdropweights-improvement-18-0.99.hdf5')

In [0]:
nnpred=keras_model2.predict(sequences_ori)

In [0]:
snn= sentences.reset_index()

In [164]:
snn.shape, nnpred.shape , sentences.shape

((168236, 2), (168236, 1), (168236,))

In [0]:
snn['pred']= nnpred

In [0]:
test_df['send_date']= pd.to_datetime(test_df.send_date,format="%d-%m-%Y %H:%M")

In [0]:
test_df['send_week']=test_df.send_date.dt.week
test_df['send_day']= test_df.send_date.dt.day
test_df['send_hour']= test_df.send_date.dt.hour
test_df['send_hour']=(test_df.send_hour/6).astype('int')
test_df['send_weekday']=test_df.send_date.dt.weekday
# pred_nn=test_df.merge(snn,how='left').groupby(['campaign_id','send_weekday']).pred.apply(lambda x: x.fillna(x.mean()))

In [0]:
pred_nn=test_df.merge(snn,how='left').groupby(['campaign_id','send_weekday',
                                               'send_hour']).pred.apply(lambda x: x.fillna((x.quantile(0.9))))

In [0]:
submission= pd.read_csv('sample_submission_4fcZwvQ.csv')
submission.is_click = pred_nn
submission.to_csv('submission_lstm.csv',index=False)